In [1]:
import pandas as pd
import numpy as np
import string
import re
import math

In [30]:
file_path = 'data/NIKO_data.xlsx'
df = pd.read_excel(file_path)

In [31]:
column_names = ['Number', 'Class', 'Sex', 'Rus', 'Math', 'Family', 'Pride', 'Traditions', 'Activities', 'Temp', 'Region', 'Sights', 'Improvements', 'Victories', 'Personality']
df.columns = column_names
df = df[['Number', 'Class', 'Sex', 'Rus', 'Math', 'Family', 'Traditions', 'Activities', 'Region', 'Sights', 'Victories', 'Personality']]
temp = df.copy()

In [33]:
federal_districts_mapping = {
    'Центральный федеральный округ': ['Белгородская область', 'Брянская область', 'Владимирская область', 'Воронежская область', 'Калужская область', 'Костромская область', 'Курская область', 'Липецкая область', 'Орловская область', 'Рязанская область', 'Тамбовская область', 'Тверская область', 'Тульская область', 'Ярославская область'],
    'Северо-Западный федеральный округ': ['г. Санкт-Петербург', 'Республика Карелия', 'Республика Коми', 'Калининградская область', 'Мурманская область', 'Ненецкий автономный округ'],
    'Южный федеральный округ': ['Республика Калмыкия', 'Краснодарский край'],
    'Северо-Кавказский федеральный округ': ['Республика Дагестан', 'Кабардино-Балкарская Республика', 'Республика Северная Осетия - Алания'],
    'Приволжский федеральный округ': ['Республика Марий Эл', 'Республика Татарстан (Татарстан)', 'Удмуртская Республика', 'Кировская область', 'Нижегородская область', 'Пензенская область', 'Пермский край', 'Самарская область', 'Ульяновская область'],
    'Уральский федеральный округ': ['Курганская область', 'Свердловская область', 'Тюменская область', 'Ямало-Ненецкий автономный округ'],
    'Сибирский федеральный округ': ['Республика Алтай', 'Республика Тыва', 'Иркутская область', 'Кемеровская область', 'Новосибирская область', 'Омская область', 'Томская область'],
    'Дальневосточный федеральный округ': ['Республика Бурятия', 'Республика Саха (Якутия)', 'Приморский край', 'Хабаровский край', 'Амурская область', 'Сахалинская область', 'Забайкальский край', 'Еврейская автономная область']
}
for federal_district, regions in federal_districts_mapping.items():
    df.loc[df['Region'].isin(regions), 'Region'] = federal_district

In [34]:
df = df.applymap(lambda x: ''.join(char for char in str(x) if char not in string.punctuation) if pd.notna(x) else x)
df = df.applymap(lambda x: np.nan if isinstance(x, str) and not x.strip() else x)

In [35]:
df = df[df.isnull().sum(axis=1) <= 2]

In [36]:
df = df[df['Family'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [37]:
df = df[df['Traditions'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [38]:
df = df[df['Activities'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [39]:
df = df[df['Sights'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [40]:
df = df[df['Victories'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [41]:
df = df[df['Personality'].apply(lambda x: isinstance(x, str) and any(char.isalpha() and not char.isascii() for char in x))]

In [42]:
df['Number'] = df['Number'].astype(int)
df = pd.merge(temp, df, on='Number', how='inner')
df = df.drop(df.columns[13:], axis=1)
df = df.drop_duplicates(subset='Number')
df = df.reset_index(drop=True)

In [45]:
df = df[(df['Rus'] != 'нет отметки') & (df['Math'] != 'нет отметки')]
df['Rus'] = df['Rus'].astype(int)
df['Math'] = df['Math'].astype(int)

In [46]:
df['Region'] = df['Region'].replace('Северо-Кавказский федеральный округ', 'Южный федеральный округ')
df['Region'] = df['Region'].replace('Южный федеральный округ', 'Юг+Северный Кавказ')

In [47]:
df.to_excel('clear_data.xlsx')